In [16]:
import pandas as pd

In [17]:
# Input dataset
CSV_IN = "../../datasource/Deliverable1Dataset.csv"
CSV_OUT = "../../output/1_handle_missing_data/total_spent_cleaned.csv"
# Define columns name
TOTAL_SPENT = "Total Spent"
PRICE_PER_UNIT = "Price Per Unit"
QUANTITY = "Quantity"
CATEGORY = "Category"
PAYMENT_METHOD = "Payment Method"
LOCATION = "Location"
ITEM = "Item"
TRANSACTION_ID = "Transaction ID"

# Define error
COERCE_ERRORS = "coerce"

In [18]:
df = pd.read_csv(CSV_IN)

### Determine missingness of Total Spent

Before checking, convert all the existing data types (string) to appropriate numeric type for comparison the relationship since `Price Per Unit`, `Quantity` and `Total Spent` are related. Check to see how many missing observations for `Total Spent`


In [19]:
# Convert column's values to numeric, coercing errors to NaN
# col is each of the relevant columns
for col in [PRICE_PER_UNIT, QUANTITY, TOTAL_SPENT]:
    # pd.to_numeric converts values in col to numeric, with errors coerced to NaN
    # df[col] accesses the target column
    # errors=COERCE_ERRORS specifies that parsing errors are set to NaN
    df[col] = pd.to_numeric(df[col], errors=COERCE_ERRORS)


totalRow = len(df)

missingValue = df[TOTAL_SPENT].isna().sum()
# missingPercent is the percentage of rows where TOTAL_SPENT is NaN
# missingValue / totalRow computes the fraction of missing values
missingPercent = round(missingValue / totalRow * 100, 2)

print(f"Total rows: {totalRow}")
print(f"Missing {TOTAL_SPENT}: {missingValue} ({missingPercent}%)")

Total rows: 12575
Missing Total Spent: 604 (4.8%)


### Missingness mechanism

Quantifying how often `Total Spent` is missing within each `Category`, `payment method`, and `location` to see their relationships with Total Spent if they were depended on or related to each other. From that can conclude what kind of missing data type like Missing Completely At Random (MCAR), Missing At Random (MAR), or Missing Not At Random (MNAR) and propose the appropriate approach to handle


In [20]:
# Missingness by Category
byCategory = df.groupby(CATEGORY)[TOTAL_SPENT].apply(lambda x: x.isna().mean()).sort_values(ascending=False) * 100
print("\nMissing Total Spent by Category (%):")
print(byCategory.round(2))

# Missingness by Payment Method
byPayment = df.groupby(PAYMENT_METHOD)[TOTAL_SPENT].apply(lambda x: x.isna().mean()).sort_values(ascending=False) * 100
print("\nMissing Total Spent by Payment Method (%):")
print(byPayment.round(2))

# Missingness by Location
byLocation = df.groupby(LOCATION)[TOTAL_SPENT].apply(lambda x: x.isna().mean()).sort_values(ascending=False) * 100
print("\nMissing Total Spent by Location (%):")
print(byLocation.round(2))



Missing Total Spent by Category (%):
Category
Patisserie                            5.69
Computers and electric accessories    5.20
Food                                  5.10
Electric household essentials         4.71
Butchers                              4.59
Beverages                             4.53
Milk Products                         4.48
Furniture                             4.15
Name: Total Spent, dtype: float64

Missing Total Spent by Payment Method (%):
Payment Method
Digital Wallet    4.90
Cash              4.80
Credit Card       4.71
Name: Total Spent, dtype: float64

Missing Total Spent by Location (%):
Location
In-store    5.11
Online      4.50
Name: Total Spent, dtype: float64


### Co-missingness analysis

Examining whether `Total Spent` is missing alongside other key fields which determine the appropriate patterns. 


In [21]:
# related missingness with Quantity
relateMissingQty = ((df[TOTAL_SPENT].isna()) & (df[QUANTITY].isna())).sum()

print(f'Rows with both Total Spent and Quantity missing: {relateMissingQty}')
print(f'Total Spent missing: {missingValue}')
print(f'Perfect overlap: {relateMissingQty == missingValue}')

# Check co-missingness with Price Per Unit
relateMissingPrice = ((df[TOTAL_SPENT].isna()) & (df[PRICE_PER_UNIT].isna())).sum()
print(f'\nRows with both Total Spent and Price Per Unit missing: {relateMissingPrice}')

# Check co-missingness with Item
relatedMissingItem = ((df[TOTAL_SPENT].isna()) & (df[ITEM].isna())).sum()
print(f'Rows with both Total Spent and Item missing: {relatedMissingItem}')
print(f'Percentage of Total Spent missing cases with Item also missing: {relatedMissingItem / missingValue * 100:.2f}%')


Rows with both Total Spent and Quantity missing: 604
Total Spent missing: 604
Perfect overlap: True

Rows with both Total Spent and Price Per Unit missing: 0
Rows with both Total Spent and Item missing: 604
Percentage of Total Spent missing cases with Item also missing: 100.00%


### Reconstructability assessment

Since `Total Spent = Price Per Unit × Quantity`, we assess how many missing `Total Spent` values could theoretically be reconstructed from the other two fields. This determines whether imputation is feasible or deletion is necessary.


In [22]:
# Check if Total Spent can be reconstructed from Price Per Unit and Quantity
# For reconstruction, we need Total Spent to be missing BUT both Price and Quantity to be present
reconstructable = df[TOTAL_SPENT].isna() & df[PRICE_PER_UNIT].notna() & df[QUANTITY].notna()
reconstructableCount = reconstructable.sum()

print(f'Missing Total Spent that CAN be reconstructed: {reconstructableCount} out of {missingValue}')
print(f'Reconstruction rate: {reconstructableCount / missingValue:.1%}')


Missing Total Spent that CAN be reconstructed: 0 out of 604
Reconstruction rate: 0.0%


### Missing data classification

**Classification: MAR (Missing At Random)**

**Rationale:**
- Perfect co-missingness with `Quantity` (604 cases = 100% overlap)
- Missing rates vary by category (4.15%-5.69%), indicating dependence on observable characteristics
- Strongly correlated with `Item` field missingness (100% of cases have Item also missing)
- The missingness is systematic and related to the Item field (an observable variable)
- Not MCAR because missing rates are not uniform across categories
- Not MNAR because the missingness is explained by observable variables (Item field status)

**Key finding:** When `Item` was not recorded during data collection, both `Quantity` and `Total Spent` were also systematically omitted, suggesting a data entry workflow issue rather than values being hidden due to their magnitude.


### Handling strategy: Listwise deletion

**Justification for deletion (not imputation):**

1. **Critical target variable:** `Total Spent` is essential for transaction analysis and should not be estimated
2. **Perfect co-missingness:** These 604 rows also have missing `Quantity`, making reconstruction impossible
3. **Cannot reliably impute:** Missing both Quantity AND at least one other key field
4. **Small data loss:** Only 4.8% of the dataset vs. large gain in data integrity
5. **Side benefit:** Eliminates 604 problematic cases that would require multiple imputations

**Alternative considered:** Reconstruct using Price × Quantity
- Not feasible: 0% of missing Total Spent cases have both Price and Quantity present
- Would require imputing Quantity first, introducing estimation error into a critical field


In [23]:

# Use mask to filter rows, then select columns
print(df[df[TOTAL_SPENT].isna()][[TRANSACTION_ID, CATEGORY, ITEM, PRICE_PER_UNIT, QUANTITY, TOTAL_SPENT]].head(10))


    Transaction ID                            Category Item  Price Per Unit  \
6      TXN_1005543                                Food  NaN            30.5   
64     TXN_1041483       Electric household essentials  NaN            15.5   
65     TXN_1041890                           Furniture  NaN            27.5   
104    TXN_1069238                                Food  NaN             5.0   
180    TXN_1130015                       Milk Products  NaN             9.5   
216    TXN_1153995       Electric household essentials  NaN            23.0   
217    TXN_1154680                           Furniture  NaN            35.0   
225    TXN_1158381                            Butchers  NaN            36.5   
249    TXN_1175914                            Butchers  NaN            23.0   
262    TXN_1187836  Computers and electric accessories  NaN            38.0   

     Quantity  Total Spent  
6         NaN          NaN  
64        NaN          NaN  
65        NaN          NaN  
104       NaN 

In [24]:
# Count rows before deletion
rowsBefore = len(df)
print(f'Rows before deletion: {rowsBefore}')
print(f'Rows to be deleted: {missingValue}')

# Perform listwise deletion
dataCleaned = df.dropna(subset=[TOTAL_SPENT])

# Count rows after deletion
rowsAfter = len(dataCleaned)
# Calculate retention rate as percentage
retentionRate = (rowsAfter / rowsBefore) * 100

print(f'\nRows after deletion: {rowsAfter}')
print(f'Rows deleted: {rowsBefore - rowsAfter}')


Rows before deletion: 12575
Rows to be deleted: 604

Rows after deletion: 11971
Rows deleted: 604


### Validation: Side benefits of deletion

Verify that deleting rows with missing `Total Spent` also eliminates other missing value problems, particularly with `Quantity`.


In [25]:
# Iterate through critical numeric columns
for col in [TOTAL_SPENT, QUANTITY, PRICE_PER_UNIT, ITEM]:
    # dataCleaned[col].isna().sum() counts missing values in each column
    missing_count = dataCleaned[col].isna().sum()
    # Calculate percentage of missing values
    missing_pct = (missing_count / len(dataCleaned)) * 100
    # f-string formatting: {col:20s} pads column name to 20 characters for alignment
    print(f'{col:20s}: {missing_count:5d} ({missing_pct:5.2f}%)')

# Verify Total Spent is now complete
print(f'\n✓ Total Spent is now 100% complete: {dataCleaned[TOTAL_SPENT].isna().sum() == 0}')


Total Spent         :     0 ( 0.00%)
Quantity            :     0 ( 0.00%)
Price Per Unit      :   609 ( 5.09%)
Item                :   609 ( 5.09%)

✓ Total Spent is now 100% complete: True


### Mathematical consistency check

Verify that for all complete rows, the relationship `Total Spent = Price Per Unit × Quantity` holds true. This validates data quality and ensures no mathematical inconsistencies exist.


In [26]:

completeRows = dataCleaned[[PRICE_PER_UNIT, QUANTITY, TOTAL_SPENT]].notna().all(axis=1)
completeData = dataCleaned[completeRows].copy()

print(f'Rows with complete Price, Quantity, and Total Spent: {len(completeData)}')

# Calculate expected Total Spent using the formula
completeData['Calculated Total'] = completeData['Price Per Unit'] * completeData['Quantity']

# Calculate absolute difference between actual and calculated
completeData['Difference'] = abs(completeData['Total Spent'] - completeData['Calculated Total'])

# Count rows with significant differences (> 0.01 to account for floating point precision)
inconsistent = (completeData['Difference'] > 0.01).sum()

print(f'Rows with mathematical inconsistency (diff > 0.01): {inconsistent}')



Rows with complete Price, Quantity, and Total Spent: 11362
Rows with mathematical inconsistency (diff > 0.01): 0


### Impact on remaining missing values

Analyze how deletion of Total Spent missing rows affects the overall missing data landscape, particularly for `Item` and `Price Per Unit` which will be handled in subsequent steps.


In [27]:
# Count missing Item values before and after deletion
itemMissingBefore = df[ITEM].isna().sum()
itemMissingAfter = dataCleaned[ITEM].isna().sum()
itemRemoved = itemMissingBefore - itemMissingAfter
itemRemovedPct = (itemRemoved / itemMissingBefore * 100) if itemMissingBefore else 0

print(f'Item missing before: {itemMissingBefore}')
print(f'Item missing after: {itemMissingAfter}')
print(f'Item missing rows removed: {itemRemoved} ({itemRemovedPct:.1f}% of Item missing cases)')
print(f'Item missing rows remaining: {itemMissingAfter} ({(itemMissingAfter / itemMissingBefore * 100) if itemMissingBefore else 0:.1f}%)')

print('\nImpact on Price Per Unit missingness:')
print('=' * 80)
# Count missing Price Per Unit values before and after deletion
priceMissingBefore = df[PRICE_PER_UNIT].isna().sum()
priceMissingAfter = dataCleaned[PRICE_PER_UNIT].isna().sum()
priceRemoved = priceMissingBefore - priceMissingAfter
priceRemovedPct = (priceRemoved / priceMissingBefore * 100) if priceMissingBefore else 0

print(f'Price Per Unit missing before: {priceMissingBefore}')
print(f'Price Per Unit missing after: {priceMissingAfter}')


Item missing before: 1213
Item missing after: 609
Item missing rows removed: 604 (49.8% of Item missing cases)
Item missing rows remaining: 609 (50.2%)

Impact on Price Per Unit missingness:
Price Per Unit missing before: 609
Price Per Unit missing after: 609


### Persist results

Save the cleaned dataset with Total Spent missing rows removed. This becomes the input for subsequent imputation steps (Price Per Unit, then Item).


In [28]:
# Save cleaned dataset to CSV
dataCleaned.to_csv(CSV_OUT, index=False)


### Summary

**Total Spent Handling**

**Classification:** MAR (Missing At Random)
- Missingness depends on Item field (observable variable)
- Perfect co-missingness with Quantity (100% overlap)

**Method:** Listwise deletion
- Removed 604 rows (4.8% of dataset)
- Retained 95.2% of data

**Justification:**
- Total Spent is a critical target variable that should not be estimated
- Cannot reconstruct: all missing cases also lack Quantity
- Small data loss with large gain in data integrity
